In [131]:
test1 = "o/ 근데 (70%)/(칠십 퍼센트)가 커 보이긴 하는데 (200)/(이백) 벌다 (140)/(백 사십) 벌면 빡셀걸? b/"
test2 = "근데 (3학년)/(삼 학년) 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?"

def bracket_filter(sentence):
  new_sentence = str()
  flag = False
  
  for ch in sentence:
    if ch == '(' and flag == False:
      flag = True
      continue
    if ch == '(' and flag == True:
      flag = False
      continue
    if ch != ')' and flag == False:
      new_sentence += ch
  return new_sentence

print(bracket_filter(test1))
print(bracket_filter(test2))


o/ 근데 칠십 퍼센트가 커 보이긴 하는데 이백 벌다 백 사십 벌면 빡셀걸? b/
근데 삼 학년 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?


In [132]:
import re

test1 = "o/ 근데 칠십 퍼센트가 커 보이긴 하는데 이백 벌다 백 사십 벌면 빡셀걸? b/"
test2 = "c# 배워봤어?"

def special_filter(sentence):
  SENTENCE_MARK = ['?', '!']
  NOISE = ['o', 'n', 'u', 'b', 'l']
  EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', '.', ',']
  
  new_sentence = str()
  for idx, ch in enumerate(sentence):
    if ch not in SENTENCE_MARK:
      # o/, n/ 등 처리
      if idx + 1 < len(sentence) and ch in NOISE and sentence[idx+1] == '/': 
        continue 

    if ch == '#': 
      new_sentence += '샾'

    elif ch not in EXCEPT: 
      new_sentence += ch

  pattern = re.compile(r'\s\s+')
  new_sentence = re.sub(pattern, ' ', new_sentence.strip())
  return new_sentence

print(special_filter(test1))
print(special_filter(test2))


근데 칠십 퍼센트가 커 보이긴 하는데 이백 벌다 백 사십 벌면 빡셀걸?
c샾 배워봤어?


In [133]:
# test = "o/ 근데 (70%)/(칠십 퍼센트)가 커 보이긴 하는데 (200)/(이백) 벌다 (140)/(백 사십) 벌면 빡셀걸? b/"
test = 'b/ 아 나 (3DS)/(쓰리 DS) 갖고 싶다. (3DS)/(쓰리 DS)'

def sentence_filter(raw_sentence):
  return special_filter(bracket_filter(raw_sentence))

print(sentence_filter(test))


아 나 쓰리 DS 갖고 싶다 쓰리 DS


In [134]:
chosung = ("ㄱ", "ㄲ", "ㄴ", "ㄷ", "ㄸ", "ㄹ", "ㅁ", "ㅂ", "ㅃ", "ㅅ", "ㅆ", "ㅇ", "ㅈ", "ㅉ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ")

jungsung = ("ㅏ", "ㅐ", "ㅑ", "ㅒ", "ㅓ", "ㅔ", "ㅕ", "ㅖ", "ㅗ", "ㅘ", "ㅙ", "ㅚ", "ㅛ", "ㅜ", "ㅝ", "ㅞ", "ㅟ", "ㅠ", "ㅡ", "ㅢ", "ㅣ")

jongsung = ("", "ㄱ", "ㄲ", "ㄳ", "ㄴ", "ㄵ", "ㄶ", "ㄷ", "ㄹ", "ㄺ", "ㄻ", "ㄼ", "ㄽ", "ㄾ", "ㄿ", "ㅀ", "ㅁ", "ㅂ", "ㅄ", "ㅅ", "ㅆ", "ㅇ", "ㅈ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ")

def isHangeul(one_character):
    return 0xAC00 <= ord(one_character[:1]) <= 0xD7A3

def hangeulExplode(one_hangeul):
    a = one_hangeul[:1]
    if isHangeul(a) != True:
        return False
    b = ord(a) - 0xAC00
    cho = b // (21*28)
    jung = b % (21*28) // 28
    jong = b % 28
    if jong == 0:
        return (chosung[cho], jungsung[jung])
    else:
        return (chosung[cho], jungsung[jung], jongsung[jong])

def hangeulJoin(inputlist):
    result = ""
    cho, jung, jong = 0, 0, 0
    inputlist.insert(0, "")
    while len(inputlist) > 1:
        if inputlist[-1] in jongsung:
            if inputlist[-2] in jungsung:
                jong = jongsung.index(inputlist.pop())
            
            else:
                result += inputlist.pop()
        elif inputlist[-1] in jungsung:
            if inputlist[-2] in chosung:
                jung = jungsung.index(inputlist.pop())
                cho = chosung.index(inputlist.pop())
                result += chr(0xAC00 + ((cho*21)+jung)*28+jong)
                cho, jung, jong = 0, 0, 0
            else:
                result += inputlist.pop()

        else:
            result += inputlist.pop()
    else:
        return result[::-1]

def pureosseugi(inputtext):
    result = ""
    for i in inputtext:
        if isHangeul(i) == True:
            for j in hangeulExplode(i):
                result += j
        else:
            result += i
    
    return result

def moasseugi(inputtext):
    t1 = []
    for i in inputtext:
        t1.append(i)

    return hangeulJoin(t1)


In [135]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

class Phoneme:
  def __init__ (self):
    self.BLANK_INDEX = 0
    self.START_INDEX = 1
    self.END_INDEX = 2
    self.SPACE_INDEX = 3
    self.OOV_INDEX = 4

class PhonemeGenerator (Phoneme):
  def __init__ (self, labels):
    super(PhonemeGenerator, self).__init__()
    self.phonemes = ['<BLANK>', '<START>', '<END>', ' ']

    texts = []
    for label in labels:
      for phoneme in label:
        if not isHangeul(phoneme) and not phoneme == ' ' and not phoneme == '?':
          break
        for pure in pureosseugi(phoneme):
          texts.append(pure)

    tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')
    tokenizer.fit_on_texts(texts)

    for i in range(1, len(tokenizer.index_word) + 1):
      self.phonemes.append(tokenizer.index_word[i])

    self.phoneme_index = {}
    for i, phoneme in enumerate(self.phonemes):
      self.phoneme_index[phoneme] = i
    
    self.VOCAB_SIZE = len(self.phonemes)
  
  def save (self, filename):
    with open(filename, 'wb') as f:
      pickle.dump((self.phonemes, self.phoneme_index), f)


class PhonemeLoader (Phoneme):
  def __init__ (self, filename):
    super(PhonemeLoader, self).__init__()
    with open(filename, 'rb') as f:
      self.phonemes, self.phoneme_index = pickle.load(f)

    self.VOCAB_SIZE = len(self.phonemes)


In [136]:
import random
import os
import numpy as np
import librosa
import copy

class Dataset:
  def get_data (self, idx):
    return (
      self.extract_spectrogram(self.file_phoneme_vector_pairs[idx][0]),
      [0] + self.file_phoneme_vector_pairs[idx][1]
    )
  
  def get_len (self):
    return len(self.file_phoneme_vector_pairs)

  def extract_spectrogram (self, filename):
    waveform = np.memmap(filename, dtype='h', mode='r')

    sr = 16000
    st = 1 / sr

    t = np.linspace(0, waveform.shape[0] * st, waveform.shape[0])

    S = librosa.feature.melspectrogram(y=np.float32(waveform), n_fft=512, hop_length=256, window='hamming', sr=sr, n_mels=128)
    log_S = librosa.power_to_db(S, ref=np.max)
    log_S = np.transpose(log_S)

    return log_S

class ShufflableDataset (Dataset):
  def __init__ (self, phoneme, file_phoneme_vector_pairs, original_file_phoneme_vector_pairs):
    super(ShufflableDataset, self).__init__()
    self.phoneme = phoneme
    self.original_file_phoneme_vector_pairs = original_file_phoneme_vector_pairs
    self.file_phoneme_vector_pairs = file_phoneme_vector_pairs
  
  def shuffle (self):
    self.file_phoneme_vector_pairs = copy.deepcopy(self.original_file_phoneme_vector_pairs)
    random.shuffle(self.file_phoneme_vector_pairs)
  
  def save (self, filename):
    with open(filename, 'wb') as f:
      pickle.dump((self.phoneme, self.file_phoneme_vector_pairs, self.original_file_phoneme_vector_pairs), f)

class ShufflableDatasetGenerator:
  @staticmethod
  def generate ():
    with open('./ko-audio-dataset/sentence-script/KsponSpeech_scripts/train.trn', 'r', encoding='utf-8') as file:
      lines = file.readlines()
    def process_line (line):
      line = line.split(' :: ')
      line[0] = os.path.join('./ko-audio-dataset/audio/', line[0])
      line[1] = sentence_filter(line[1])
      return line
    file_label_pairs = list(map(process_line, lines))
    file_label_pairs = list(filter(lambda pair: not ShufflableDatasetGenerator.check_skip_label(pair[1]), file_label_pairs))
    phoneme_generator = PhonemeGenerator([label for file, label in file_label_pairs])
    file_phoneme_pairs = list(map(lambda pair: (pair[0], pureosseugi(pair[1])), file_label_pairs))
    original_file_phoneme_vector_pairs = [[pair[0], [phoneme_generator.phoneme_index[phoneme] for phoneme in pair[1]]] for pair in file_phoneme_pairs]
    file_phoneme_vector_pairs = copy.deepcopy(original_file_phoneme_vector_pairs)
    return ShufflableDataset(phoneme_generator, file_phoneme_vector_pairs, original_file_phoneme_vector_pairs)

  @staticmethod
  def check_skip_hangul (hangul):
    return not isHangeul(hangul) and not hangul == ' ' and not hangul == '?'

  @staticmethod
  def check_skip_label (label):
    for hangul in label:
      if ShufflableDatasetGenerator.check_skip_hangul(hangul):
        return True
    return False

class ShufflableDatasetLoader:
  @staticmethod
  def load (filename):
    with open(filename, 'rb') as f:
      phoneme, file_phoneme_vector_pairs, original_file_phoneme_vector_pairs = pickle.load(f)
    return ShufflableDataset(phoneme, file_phoneme_vector_pairs, original_file_phoneme_vector_pairs)


In [137]:
class DatasetSizeLimitingIterator:
  def __init__ (self, dataset, time_limit, label_limit, idx):
    self.dataset = dataset
    self.time_limit = time_limit
    self.label_limit = label_limit
    self.idx = idx
    while self.idx < self.dataset.get_len():
      data = self.dataset.get_data(self.idx)
      if data[0].shape[0] <= self.time_limit and len(data[1]) <= self.label_limit:
        break
      self.idx += 1

  def next (self):
    idx = self.idx + 1
    while idx < self.dataset.get_len():
      data = self.dataset.get_data(idx)
      if data[0].shape[0] <= self.time_limit and len(data[1]) <= self.label_limit:
        break
      idx += 1
    return DatasetSizeLimitingIterator(self.dataset, self.time_limit, self.label_limit, idx)
  
  def check_end (self):
    return self.idx >= self.dataset.get_len()

  def get_value (self):
    return self.dataset.get_data(self.idx)

  def save (self, filename):
    with open(filename, 'wb') as f:
      pickle.dump((self.time_limit, self.label_limit,self.idx), f)

class IteratorLoader:
  @staticmethod
  def load (dataset, filename):
    with open(filename, 'rb') as f:
      time_limit, label_limit, idx = pickle.load(f)
    return DatasetSizeLimitingIterator(dataset, time_limit, label_limit, idx)


In [138]:
import os

class DatasetStepExtractorLoader:
  @staticmethod
  def load (path):
    dataset = ShufflableDatasetLoader.load(os.path.join(path, 'dataset.pickle'))
    return SavableDatasetStepExtractor(
      dataset,
      pickle.load(open(os.path.join(path, 'step_size.pickle'), 'rb')),
      IteratorLoader.load(dataset, os.path.join(path, 'iterator.pickle'))
    )

class DatasetStepExtractor:
  def __init__ (self, dataset, step_size, iterator):
    self.dataset = dataset
    self.step_size = step_size
    self.value = []
    self.iterator = iterator
    while not iterator.check_end() and len(self.value) < self.step_size:
      self.value.append(iterator.get_value())
      iterator = iterator.next()
    self.is_end = len(self.value) < self.step_size
    self.next_iterator = iterator
  
  def check_end (self):
    return self.is_end
  
  def get_value (self):
    return self.value
  
  def get_next_extractor (self):
    if self.is_end:
      return None
    else:
      return SavableDatasetStepExtractor(self.dataset, self.step_size, self.next_iterator)

class SavableDatasetStepExtractor (DatasetStepExtractor):
  def save (self, saver):
    saver.save(self.dataset, self.step_size, self.iterator)

class DatasetStepExtractorSaver:
  def __init__ (self, path):
    self.path = path
  
  def save (self, dataset, step_size, iterator):
    os.makedirs(self.path, exist_ok=True)

class IteratorSaver (DatasetStepExtractorSaver):
  def save (self, dataset, step_size, iterator):
    super().save(dataset, step_size, iterator)
    iterator.save(os.path.join(self.path, 'iterator.pickle'))
  
class DatasetSaver (DatasetStepExtractorSaver):
  def save (self, dataset, step_size, iterator):
    super().save(dataset, step_size, iterator)
    dataset.save(os.path.join(self.path, 'dataset.pickle'))

class StepSizeSaver (DatasetStepExtractorSaver):
  def save (self, dataset, step_size, iterator):
    super().save(dataset, step_size, iterator)
    with open(os.path.join(self.path, 'step_size.pickle'), 'wb') as f:
      pickle.dump(step_size, f)

class ExtractorSaver (DatasetStepExtractorSaver):
  def save (self, dataset, step_size, iterator):
    super().save(dataset, step_size, iterator)
    iterator.save(os.path.join(self.path, 'iterator.pickle'))
    dataset.save(os.path.join(self.path, 'dataset.pickle'))
    with open(os.path.join(self.path, 'step_size.pickle'), 'wb') as f:
      pickle.dump(step_size, f)


In [139]:
################ 여기 까지 dataset 부분 ###############

In [140]:
import tensorflow as tf
import numpy as np

In [141]:
def nan_to_zero (tensor):
  return tf.where(tf.math.is_nan(tensor), tf.zeros_like(tensor), tensor)

def rnnt_loss (logits, labels, time_lengths, label_lengths):
  log_pr = tf.math.log_softmax(logits, axis=-1)
  pr = pr_loss(log_pr, labels, time_lengths, label_lengths)
  ret = tf.reduce_sum(pr)
  return ret

@tf.custom_gradient
def pr_loss (log_pr, labels, time_lengths, label_lengths):
  LOG_0 = float('-inf')
  batch_size = tf.shape(log_pr)[0]
  max_time_lengths = tf.shape(log_pr)[1]
  max_label_lengths = tf.shape(log_pr)[2]

  def get_truth_log_pr (log_pr, labels):
    labels_one_hot = tf.one_hot(labels, tf.shape(log_pr)[-1], axis=-1, dtype=tf.float32)
    labels_one_hot = labels_one_hot[:, 1: ]
    labels_one_hot = tf.expand_dims(labels_one_hot, axis=1)
    labels_one_hot = tf.repeat(labels_one_hot, tf.shape(log_pr)[1], axis=1)
    ret = tf.reduce_sum(log_pr[:, :, : -1, :] * labels_one_hot, axis=-1)
    ret = tf.concat([
      ret,
      LOG_0 * tf.ones((tf.shape(log_pr)[0], tf.shape(log_pr)[1], 1), dtype=tf.float32)
    ], axis=-1)
    return ret
  
  def get_blank_log_pr (log_pr):
    return log_pr[:, :, :, 0]

  truth_log_pr = get_truth_log_pr(log_pr, labels)
  blank_log_pr = get_blank_log_pr(log_pr)

  def get_alpha (truth_log_pr, blank_log_pr):
    reversed_truth_log_pr = tf.reverse(truth_log_pr, axis=[-1])
    padded_truth_log_pr = tf.pad(
      reversed_truth_log_pr,
      [[0, 0], [0, 0], [tf.shape(reversed_truth_log_pr)[-2] - 1, 0]],
      constant_values=LOG_0
    )
    truth_diag = tf.linalg.diag_part(
      padded_truth_log_pr,
      k=(0, tf.shape(padded_truth_log_pr)[-1] - 1),
      padding_value=LOG_0,
      align='LEFT_RIGHT'
    )
    truth_diag = tf.transpose(truth_diag, perm=[1, 0, 2])

    reversed_blank_log_pr = tf.reverse(blank_log_pr, axis=[-1])
    padded_blank_log_pr = tf.pad(
      reversed_blank_log_pr,
      [[0, 0], [0, 0], [tf.shape(reversed_blank_log_pr)[-2] - 1, 0]],
      constant_values=LOG_0
    )
    blank_diag = tf.linalg.diag_part(
      padded_blank_log_pr,
      k=(0, tf.shape(padded_blank_log_pr)[-1] - 1),
      padding_value=LOG_0,
      align='LEFT_RIGHT'
    )
    blank_diag = tf.concat([
      LOG_0 * tf.ones((tf.shape(blank_diag)[0], tf.shape(blank_diag)[1], 1), dtype=tf.float32),
      blank_diag[:, :, : -1]
    ], axis=-1)
    blank_diag = tf.transpose(blank_diag, perm=[1, 0, 2])

    initial_diag = tf.concat([
      tf.zeros((tf.shape(blank_diag)[1], 1), dtype=tf.float32),
      LOG_0 * tf.ones((tf.shape(blank_diag)[1], tf.shape(blank_diag)[-1] - 1), dtype=tf.float32)
    ], axis=-1)

    def step (a, x):
      t, b = x
      return (
        tf.reduce_logsumexp(
          tf.stack([
              a + t,
              tf.concat([
                LOG_0 * tf.ones((tf.shape(a)[0], 1), dtype=tf.float32),
                a[:, : -1]
              ], axis=-1) + b
            ],
            axis=0
          ), axis=0
        )
      )
    alpha_diag =  tf.concat([
      tf.expand_dims(initial_diag, axis=0),
      tf.scan(step, (truth_diag, blank_diag), initial_diag)
    ], axis=0)
    alpha_diag = tf.transpose(alpha_diag, perm=[1, 2, 0])
    alpha = tf.linalg.diag_part(alpha_diag, k=(0, max_label_lengths - 1))
    alpha = tf.reverse(alpha, axis=[-2])
    alpha = tf.transpose(alpha, perm=[0, 2, 1])
    return alpha

  def get_beta (truth_log_pr, blank_log_pr):
    reversed_truth_log_pr = tf.reverse(truth_log_pr, axis=[-1])
    reversed_truth_log_pr = tf.concat([
      LOG_0 * tf.ones((tf.shape(reversed_truth_log_pr)[0], tf.shape(reversed_truth_log_pr)[1], 1), dtype=tf.float32),
      reversed_truth_log_pr[:, :, 1: ]
    ], axis=-1)
    padded_truth_log_pr = tf.pad(
      reversed_truth_log_pr,
      [[0, 0], [0, 0], [tf.shape(reversed_truth_log_pr)[-2] - 1, 0]],
      constant_values=LOG_0
    )
    truth_diag = tf.linalg.diag_part(
      padded_truth_log_pr,
      k=(0, tf.shape(padded_truth_log_pr)[-1] - 1),
      padding_value=LOG_0,
      align='LEFT_RIGHT'
    )
    truth_diag = tf.transpose(truth_diag, perm=[1, 0, 2])
    truth_diag = truth_diag[: -1]

    reversed_blank_log_pr = tf.reverse(blank_log_pr, axis=[-1])
    reversed_blank_log_pr = tf.concat([
      reversed_blank_log_pr[:, : -1, :],
      LOG_0 * tf.ones((tf.shape(reversed_blank_log_pr)[0], 1, tf.shape(reversed_blank_log_pr)[-1]), dtype=tf.float32)
    ], axis=-2)
    padded_blank_log_pr = tf.pad(
      reversed_blank_log_pr,
      [[0, 0], [0, 0], [tf.shape(reversed_blank_log_pr)[-2] - 1, 0]],
      constant_values=LOG_0
    )
    blank_diag = tf.linalg.diag_part(
      padded_blank_log_pr,
      k=(0, tf.shape(padded_blank_log_pr)[-1] - 1),
      padding_value=LOG_0,
      align='LEFT_RIGHT'
    )
    blank_diag = tf.transpose(blank_diag, perm=[1, 0, 2])
    blank_diag = blank_diag[: -1]

    mask = tf.sequence_mask(
      time_lengths + label_lengths - 2,
      tf.shape(log_pr)[1] + tf.shape(log_pr)[2] - 2,
      dtype=tf.float32
    )
    mask = tf.transpose(mask, perm=[1, 0])

    dp_start_value = tf.gather_nd(
      blank_log_pr,
      indices=tf.stack([time_lengths, label_lengths], axis=-1) - 1,
      batch_dims=1
    )

    initial_diag_mask = tf.one_hot(time_lengths - 1, depth=tf.shape(log_pr)[1])
    initial_diag = tf.expand_dims(dp_start_value, axis=1) * initial_diag_mask + nan_to_zero(LOG_0 * (1.0 - initial_diag_mask))

    def step (a, x):
      m, t, b = x
      a_next = tf.reduce_logsumexp(
        tf.stack([
          a + t,
          tf.concat([
            a[:, 1: ],
            LOG_0 * tf.ones((tf.shape(a)[0], 1), dtype=tf.float32)
          ], axis=-1) + b
        ], axis=0),
        axis=0
      )
      masked_a_next = nan_to_zero(a_next * tf.expand_dims(m, axis=1)) + nan_to_zero(a * tf.expand_dims(1.0 - m, axis=1))
      return masked_a_next

    beta_diag = tf.concat([
      tf.scan(step, (mask, truth_diag, blank_diag), initial_diag, reverse=True),
      tf.expand_dims(initial_diag, axis=0)
    ], axis=0)

    beta_diag = tf.transpose(beta_diag, perm=[1, 2, 0])
    beta = tf.linalg.diag_part(beta_diag, k=(0, tf.shape(log_pr)[2] - 1), padding_value=LOG_0)
    beta = tf.transpose(beta, perm=[0, 2, 1])
    beta = tf.reverse(beta, axis=[-1])

    return beta
  
  time_mask = tf.sequence_mask(time_lengths, tf.shape(log_pr)[1], dtype=tf.float32)
  label_mask = tf.sequence_mask(label_lengths, tf.shape(log_pr)[2], dtype=tf.float32)
  total_mask = tf.expand_dims(time_mask, axis=2) * tf.expand_dims(label_mask, axis=1)

  alpha = get_alpha(truth_log_pr, blank_log_pr)
  alpha = alpha + nan_to_zero((1.0 - total_mask) * LOG_0)
  beta = get_beta(truth_log_pr, blank_log_pr)
  beta = beta + nan_to_zero((1.0 - total_mask) * LOG_0)

  indices = tf.concat([
    tf.expand_dims(tf.range(0, tf.shape(log_pr)[0]), axis=1),
    tf.stack([
      time_lengths,
      label_lengths - 1
    ], axis=-1),
  ], axis=-1)
  
  beta_mask = tf.scatter_nd(
    indices,
    tf.ones(tf.shape(indices)[0], tf.float32),
    (tf.shape(log_pr)[0], tf.shape(log_pr)[1] + 1, tf.shape(log_pr)[2])
  )
  beta_mask = 1.0 - beta_mask
  beta = nan_to_zero(tf.pad(beta, [[0, 0], [0, 1], [0, 0]], constant_values=LOG_0) * beta_mask)

  total_mask = tf.expand_dims(total_mask, axis=-1)

  total_log_pr = beta[:, 0, 0]

  def grad (upstream):
    blank_grads = \
      alpha + beta[:, 1: , :] \
      - tf.reshape(total_log_pr, shape=(tf.shape(total_log_pr)[0], 1, 1))
    truth_grads = \
      alpha + tf.pad(
        beta[:, : -1, 1: ],
        [[0, 0], [0, 0], [0, 1]],
        constant_values=LOG_0
      ) \
      - tf.reshape(total_log_pr, shape=(tf.shape(total_log_pr)[0], 1, 1))
    blank_one_hot = tf.one_hot(tf.zeros_like(labels, dtype=tf.int32), tf.shape(log_pr)[-1], dtype=tf.float32)
    blank_one_hot = tf.expand_dims(blank_one_hot, axis=1)
    blank_one_hot = tf.repeat(blank_one_hot, tf.shape(log_pr)[1], axis=1)
    blank_grads = tf.exp(tf.expand_dims(blank_grads, axis=-1) + log_pr) * blank_one_hot
    truth_one_hot = tf.one_hot(labels[:, 1: ], tf.shape(log_pr)[-1], dtype=tf.float32)
    truth_one_hot = tf.concat([truth_one_hot, tf.zeros((tf.shape(log_pr)[0], 1, tf.shape(log_pr)[-1]), dtype=tf.float32)], axis=-2)
    truth_one_hot = tf.expand_dims(truth_one_hot, axis=1)
    truth_one_hot = tf.repeat(truth_one_hot, tf.shape(log_pr)[1], axis=1)
    truth_grads = tf.exp(tf.expand_dims(truth_grads, axis=-1) + log_pr) * truth_one_hot

    grads = blank_grads + truth_grads

    return (
      [
        tf.reshape(-upstream, shape=(tf.shape(upstream)[0], 1, 1, 1))
        * grads * total_mask
      ] +
      [None] * 3
    )

  return -total_log_pr, grad


In [142]:
EMBEDDING_DIM = 256
UNITS = 1024
CODER_OUTPUT_DIM = 512
JOINT_NET_INNER_DIM = 1024

In [143]:
class Encoder (tf.keras.Model):
  def __init__ (self, units, output_dim):
    super(Encoder, self).__init__()
    self.gru = tf.keras.layers.GRU(units, recurrent_initializer='glorot_uniform', return_sequences=True, return_state=True)
    self.fc = tf.keras.layers.Dense(output_dim)
    
  def call (self, inputs):
    rnn_outputs, hidden_states = self.gru(inputs)
    outputs = self.fc(rnn_outputs)
    return outputs


In [144]:
class Decoder (tf.keras.Model):
  def __init__ (self, vocab_size, embedding_dim, units, output_dim):
    super(Decoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(units, recurrent_initializer='glorot_uniform', return_sequences=True, return_state=True)
    self.fc = tf.keras.layers.Dense(output_dim)

  def call (self, inputs):
    embedded = self.embedding(inputs)
    outputs, hidden_states = self.gru(embedded)
    outputs = self.fc(outputs)
    return outputs, hidden_states

In [145]:
class JointNet (tf.keras.Model):
  def __init__ (self, inner_dim, vocab_size):
    super(JointNet, self).__init__()
    self.forward_layer = tf.keras.layers.Dense(inner_dim, activation='tanh')
    self.project_layer = tf.keras.layers.Dense(vocab_size)
  
  def call (self, inputs):
    enc_outputs, dec_outputs = inputs
    joint_inputs = tf.expand_dims(enc_outputs, axis=2) + tf.expand_dims(dec_outputs, axis=1)
    outputs = self.forward_layer(joint_inputs)
    outputs = self.project_layer(outputs)
    return outputs


In [146]:
class Transducer (tf.keras.Model):
  def __init__ (self, embedding_dim, units, coder_output_dim, joint_net_inner_dim, vocab_size):
    super(Transducer, self).__init__()
    self.encoder = Encoder(units, coder_output_dim)
    self.decoder = Decoder(vocab_size, embedding_dim, units, coder_output_dim)
    self.joint_net = JointNet(joint_net_inner_dim, vocab_size)
    self.vocab_size = vocab_size
  
  def call (self, inputs):
    enc_inputs, dec_inputs = inputs
    enc_outputs = self.encoder(enc_inputs)
    dec_outputs, dec_states = self.decoder(dec_inputs)
    outputs = self.joint_net([enc_outputs, dec_outputs])
    return outputs


In [147]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import numpy as np
import librosa
from functools import reduce

class Saver:
  def save (self, path, cur_epoch, cnt):
    pass

class InitialSaver:
  def initial_save (self):
    pass

class TrainPolicy:
  def train (self):
    pass

class StepPolicy (TrainPolicy):
  def __init__ (
    self,
    optimizer,
    epochs,
    batch_size,
    batch_num,
    time_limit,
    label_limit,
    step_performer,
    step_end_handlers,
    epoch_end_handlers,
    dataset,
    dataset_extractor,
    cur_epoch,
    cnt
  ):
    self.optimizer = optimizer
    self.epochs = epochs
    self.batch_size = batch_size
    self.batch_num = batch_num
    self.time_limit = time_limit
    self.label_limit = label_limit

    self.step_performer = step_performer
    self.step_end_handlers = step_end_handlers
    self.epoch_end_handlers = epoch_end_handlers

    self.dataset = dataset
    self.dataset_extractor = dataset_extractor
    self.cur_epoch = cur_epoch
    self.cnt = cnt

  def train (self):
    for i in range(self.cur_epoch, self.epochs):
      while not self.dataset_extractor.check_end():
        loss = self.step_performer.step(self.optimizer, self.batch_size, self.batch_num, self.dataset_extractor.get_value())
        self.cnt += self.batch_num * self.batch_size
        for step_end_handler in self.step_end_handlers:
          step_end_handler.handle(self.cur_epoch, self.cnt, loss, self.batch_num * self.batch_size)
        self.dataset_extractor = self.dataset_extractor.get_next_extractor()
      
      self.cur_epoch += 1
      self.cnt = 0
      self.dataset.shuffle()
      self.dataset_extractor = SavableDatasetStepExtractor(
        self.dataset,
        self.batch_size * self.batch_num,
        DatasetSizeLimitingIterator(self.dataset, self.time_limit, self.label_limit, 0)
      )
      for epoch_end_handler in self.epoch_end_handlers:
        epoch_end_handler.handle(self.epochs, self.cur_epoch, self.cnt)

class SavableStepPolicy (StepPolicy, Saver, InitialSaver):
  def __init__ (
    self,
    path,
    savable_save_notifier,
    optimizer,
    epochs,
    batch_size,
    batch_num,
    time_limit,
    label_limit,
    step_performer,
    step_end_handlers,
    epoch_end_handlers,
    dataset,
    dataset_extractor,
    cur_epoch,
    cnt,
    prev_save_time
  ):
    StepPolicy.__init__(
      self,
      optimizer,
      epochs,
      batch_size,
      batch_num,
      time_limit,
      label_limit,
      step_performer,
      step_end_handlers,
      epoch_end_handlers,
      dataset,
      dataset_extractor,
      cur_epoch,
      cnt
    )
    savable_save_notifier.savers.append(self)
    savable_save_notifier.savers.append(self.step_performer)
    self.step_end_handlers.append(
      savable_save_notifier
    )
    self.epoch_end_handlers.append(
      SavableEpochSaveNotifier(
        path,
        [
          self,
          self.step_performer,
        ],
        prev_save_time
      )
    )
  
  def save (self, path, cur_epoch, cnt):
    with open(os.path.join(path, 'progress.pickle'), 'wb') as f:
      pickle.dump((self.cur_epoch, self.cnt), f)
    self.dataset_extractor.save(IteratorSaver(os.path.join(path, 'dataset_extractor')))

  def initial_save (self, path):
    with open(os.path.join(path, 'progress.pickle'), 'wb') as f:
      pickle.dump((self.cur_epoch, self.cnt), f)
    self.dataset_extractor.save(ExtractorSaver(os.path.join(path, 'dataset_extractor')))
    with open(os.path.join(path, 'policy_meta.pickle'), 'wb') as f:
      pickle.dump((self.batch_num, self.batch_size, self.time_limit, self.label_limit), f)

class TrainStepPerformer:
  def __init__ (self, transducer):
    self.transducer = transducer

  def step (self, optimizer, batch_size, batch_num, datas):
    acc_grads = [tf.zeros_like(var) for var in self.transducer.trainable_variables]
    acc_loss = 0
    for i in range(0, len(datas), batch_size):
      batch_datas = datas[i: i + batch_size]
      label_lengths = self.get_label_lengths(batch_datas)
      label_lengths = tf.convert_to_tensor(label_lengths, dtype=tf.int32)
      labels = pad_sequences(list(map(lambda x: x[1], batch_datas)), maxlen=self.get_labels_maxlen(batch_datas), truncating='post', padding='post')
      labels = list(map(lambda x: x[1], batch_datas))
      max_label_length = reduce(lambda a, x: max(a, x), [len(label) for label in labels])
      labels = [label + [0] * (max_label_length - len(label)) for label in labels]
      labels = tf.constant(labels, dtype=tf.int32)
      spectrograms = [x[0] for x in batch_datas]
      spectrogram_lengths = self.get_spectrogram_lengths(spectrograms)
      spectrogram_lengths = tf.convert_to_tensor(spectrogram_lengths, dtype=tf.int32)
      spectrograms = tf.cast(pad_sequences(spectrograms, maxlen=self.get_spectrograms_maxlen(spectrograms), truncating='post', padding='post'), dtype=tf.float32)
      with tf.GradientTape() as tape:
        logits = self.transducer([spectrograms, labels])
        loss = rnnt_loss(
          logits,
          labels,
          spectrogram_lengths,
          label_lengths
        )
      acc_loss += loss.numpy()
      grads = tape.gradient(loss, self.transducer.trainable_variables)
      acc_grads = [acc_grad + grad for acc_grad, grad in zip(acc_grads, grads)]
    acc_grads = [acc_grad / (batch_size * batch_num) for acc_grad in acc_grads]
    optimizer.apply_gradients(zip(acc_grads, self.transducer.trainable_variables))
    
    return acc_loss / (batch_size * batch_num)
  
  def get_labels_maxlen (self, batch_datas):
    ret = 0
    for file_label_vector_pair in batch_datas:
      ret = max(ret, len(file_label_vector_pair[1]))
    return ret
  
  def get_label_lengths (self, batch_datas):
    ret = list(map(lambda x: len(x[1]), batch_datas))
    return ret
  
  def get_spectrograms_maxlen (self, spectrograms):
    ret = 0
    for spectrogram in spectrograms:
      ret = max(ret, spectrogram.shape[0])
    return ret

  def get_spectrogram_lengths (self, spectrograms):
    ret = list(map(lambda x: x.shape[0], spectrograms))
    return ret

class SavableTrainStepPerformer (TrainStepPerformer, Saver, InitialSaver):
  def __init__ (self, transducer, embedding_dim, units, coder_output_dim, joint_net_inner_dim, vocab_size):
    TrainStepPerformer.__init__(self, transducer)
    self.embedding_dim = embedding_dim
    self.units = units
    self.coder_output_dim = coder_output_dim
    self.joint_net_inner_dim = joint_net_inner_dim
    self.vocab_size = vocab_size

  def save (self, path, cur_epoch, cnt):
    self.transducer.save_weights(os.path.join(path, 'checkpoints/ep{}_no{}/ckpt'.format(cur_epoch, cnt)))

  def initial_save (self, path):
    self.transducer.save_weights(os.path.join(path, 'checkpoints/ep{}_no{}/ckpt'.format(0, 0)))
    with open(os.path.join(path, 'model_meta.pickle'), 'wb') as f:
      pickle.dump((
        self.embedding_dim,
        self.units,
        self.coder_output_dim,
        self.joint_net_inner_dim,
        self.vocab_size
      ), f)

class SavableTrainStepPerformerLoader:
  @staticmethod
  def load (path):
    with open(os.path.join(path, 'model_meta.pickle'), 'rb') as f:
      model_meta = pickle.load(f)
    with open(os.path.join(path, 'progress.pickle'), 'rb') as f:
      progress = pickle.load(f)
    transducer = Transducer(*model_meta)
    transducer([
      tf.keras.Input(shape=(None, 128), dtype=tf.float32),
      tf.keras.Input(shape=(None, ), dtype=tf.int32)
    ])
    transducer.load_weights(os.path.join(path, 'checkpoints/ep{}_no{}/ckpt'.format(progress[0], progress[1])))
    return SavableTrainStepPerformer(
      transducer,
      *model_meta
    )

class PrevSaveTime:
  def __init__ (self, prev_save_time):
    self.prev_save_time = prev_save_time
  
  def update (self, prev_save_time):
    self.prev_save_time = prev_save_time

class EpochEndHandler:
  def handle (self, epochs, cur_epoch, cnt):
    pass

class EpochPrinter (EpochEndHandler):
  def handle (self, epochs, cur_epoch, cnt):
    print('epoch: {} / {}'.format(cur_epoch, epochs))

class EpochSaveNotifier (EpochEndHandler):
  def __init__ (self, path, savers):
    super(EpochSaveNotifier, self).__init__()
    self.path = path
    self.savers = savers

  def handle (self, epochs, cur_epoch, cnt):
    for saver in self.savers:
      saver.save(self.path, cur_epoch, cnt)
  
class SavableEpochSaveNotifier (EpochSaveNotifier):
  def __init__ (self, path, savers, prev_save_time):
    super(SavableEpochSaveNotifier, self).__init__(path, savers)
    self.prev_save_time = prev_save_time
    self.savers.append(self)
  
  def save (self, path, cur_epoch, cnt):
    with open(os.path.join(path, 'prev_save_time.pickle'), 'wb') as f:
      pickle.dump(0, f)
    self.prev_save_time.update(0)

class StepEndHandler:
  def handle (self, cur_epoch, cnt, loss, step_size):
    pass

class StepPrinter (StepEndHandler):
  def handle (self, cur_epoch, cnt, loss, step_size):
    print('epoch, cnt: {}, {}'.format(cur_epoch, cnt))
    print('loss: {}'.format(loss / step_size))

class SaveNotifier (StepEndHandler):
  def __init__ (self, path, savers, save_time_delta, prev_save_time):
    super(SaveNotifier, self).__init__()
    self.path = path
    self.savers = savers
    self.save_time_delta = save_time_delta
    self.prev_save_time = prev_save_time

  def handle (self, cur_epoch, cnt, loss, step_size):
    if self.prev_save_time.prev_save_time + self.save_time_delta <= cnt:
      for saver in self.savers:
        saver.save(self.path, cur_epoch, cnt)
      self.prev_save_time.update(cnt)

class SavableSaveNotifier (SaveNotifier, Saver, InitialSaver):
  def __init__ (self, path, savers, save_time_delta, prev_save_time):
    super(SavableSaveNotifier, self).__init__(path, savers, save_time_delta, prev_save_time)
    self.savers.append(self)
  
  def save (self, path, cur_epoch, cnt):
    with open(os.path.join(path, 'prev_save_time.pickle'), 'wb') as f:
      pickle.dump(cnt, f)
  
  def initial_save (self, path):
    with open(os.path.join(path, 'prev_save_time.pickle'), 'wb') as f:
      pickle.dump(0, f)

class InitialSaveNotifier:
  def __init__ (self, path, initial_savers):
    self.path = path
    self.initial_savers = initial_savers

  def initial_save (self):
    os.makedirs(self.path, exist_ok=True)
    for initial_saver in self.initial_savers:
      initial_saver.initial_save(self.path)


In [148]:
class SimplePolicyGenerator:
  @staticmethod
  def generate (save_path):
    prev_save_time = PrevSaveTime(0)
    savable_save_notifier = SavableSaveNotifier(save_path, [], 5000, prev_save_time)
    batch_size = 16
    batch_num = 2
    step_size = batch_size * batch_num
    time_limit = 160
    label_limit = 50
    dataset = ShufflableDatasetGenerator.generate()
    dataset_extractor = SavableDatasetStepExtractor(
      dataset,
      step_size,
      DatasetSizeLimitingIterator(dataset, time_limit, label_limit, 0)
    )
    cur_epoch = 0
    cnt = 0

    model_meta = (256, 1024, 512, 1024, len(dataset_extractor.dataset.phoneme.phonemes))
    transducer = Transducer(*model_meta)
    transducer([
      tf.keras.Input(shape=(None, 128), dtype=tf.float32),
      tf.keras.Input(shape=(None, ), dtype=tf.int32)
    ])

    policy = SavableStepPolicy(
      save_path,
      savable_save_notifier,
      tf.keras.optimizers.Adam(learning_rate=0.0001),
      20,
      batch_size,
      batch_num,
      160,
      50,
      SavableTrainStepPerformer(
        transducer,
        *model_meta
      ),
      [StepPrinter()],
      [EpochPrinter()],
      dataset,
      dataset_extractor,
      cur_epoch,
      cnt,
      prev_save_time
    )
    return \
      policy, \
      InitialSaveNotifier(
        save_path, [
          policy,
          policy.step_performer,
          savable_save_notifier
        ]
      )

class SimplePolicyLoader:
  @staticmethod
  def load (path):
    with open(os.path.join(path, 'progress.pickle'), 'rb') as f:
      progress = pickle.load(f)
    dataset_extractor = DatasetStepExtractorLoader.load(os.path.join(path, 'dataset_extractor'))
    prev_save_time = PrevSaveTime(
      pickle.load(
        open(os.path.join(path, 'prev_save_time.pickle'), 'rb')
      )
    )
    policy = SavableStepPolicy(
      path,
      SavableSaveNotifier(
        path,
        [],
        5000,
        prev_save_time
      ),
      tf.keras.optimizers.Adam(learning_rate=0.0001),
      20,
      16,
      2,
      160,
      50,
      SavableTrainStepPerformerLoader.load(path),
      [StepPrinter()],
      [EpochPrinter()],
      dataset_extractor.dataset,
      dataset_extractor,
      progress[0],
      progress[1],
      prev_save_time
    )
    return policy

In [149]:
# policy, initial_save_notifier = SimplePolicyGenerator.generate('train_state4')
# initial_save_notifier.initial_save()

In [150]:
policy = SimplePolicyLoader.load('saves/train_state4')

In [151]:
# with tf.device('/gpu:0'):
#   policy.train()


In [152]:
def vector_to_sentence (x):
  return ''.join([policy.dataset_extractor.dataset.phoneme.phonemes[idx] for idx in x][1: ])


In [153]:
class AudioDecoder:
  def __init__ (self, transducer):
    self.transducer = transducer
  
  def decode (self, spectrogram):
    label = [0]
    for i in range(spectrogram.shape[0]):
      sub_spectrogram = spectrogram[0: i + 1]
      sub_spectrogram = tf.convert_to_tensor(sub_spectrogram)
      sub_spectrogram = tf.expand_dims(sub_spectrogram, axis=0)
      sub_label = tf.constant([label], dtype=tf.int32)
      sub_rst = self.transducer((sub_spectrogram, sub_label))
      # print(tf.argmax(sub_rst, axis=-1)[:, :, : ][0].numpy(), end='\n\n')
      sub_rst = tf.argmax(sub_rst[0][-1][-1], axis=-1).numpy()
      if sub_rst != 0:
        label.append(sub_rst)
    return label


In [154]:
decoder = AudioDecoder(policy.step_performer.transducer)
datas = policy.dataset_extractor.get_value()

for data in datas:
  a = list(filter(lambda x: x != 0, decoder.decode(data[0])))
  print(moasseugi(vector_to_sentence([0] + a)))
  print(moasseugi(vector_to_sentence(data[1])))
  print()

면허도 진짜 내 내
하 면허도 진짜 따야 되는데

아예 어떻게 사
아 이렇게 해서?

동생이랑
동생이랑?

으
아

그래
그래

피시 키는 승
긴 하지

응 그냥
응 그냥

내가 그러셔 먹겠었는 확은 어
데 가끔씩 막 비싼 데 가면은

그 교양이였어 강요
고 교양이었을 거 아냐

나는
아

근 진짜 입겜인 거는
근데 슈팅 게임이거든

유튜븐데 내년 조
유튜브 편집자 해야지 뭐

배는 아니야 걔가 막
백 명 단위야 꽤 많아

음 열 한 생
음 에 맞아요

뭐 연기 때
뭐 영화인데 어때?

어젠 논술 다니루고 있거든 야
노점상들 다 밀어버리니까 그냥

대전에는 좋아했거든 새
대신에 우리는 종업시를 늦게 하지

하고 놀더고 얘
강원도 동해

어
아

그럴 수 있겠다
그럴 수 있겠다

아이 곧 시워지
아니 코가 더 비싸지

그땐 그래서 그게 써질랭가
하여튼 그래서 어떻게 됐어? 뒤에가?

집에
식민

다음 주
다 먹은 줄 알

나 우리 아빠 느끈 신
나 우리 아빠 담배 끊은 게 너무 신기해

맞아
맞아

거 건 뭔가 여머도 있구
겉 겉멋만 들어가지고?

어디 치킨을 먹게 될 건
어제 치킨을 먹긴 했거든?

얘 그 지금 사 팔
얘 그리고 되게 금사빠야

그건 몸살 때야 되 못 하지 이제
그건 너무 상대한테 못 할 짓이야

뭐 시헙잖아
멋이 없잖아

아 까 나 너는 진짜
아깝다 너는 진짜



In [155]:
def decode_with_hint (logits):
  rst = []
  x, y = 0, 0
  while x < logits.shape[2] and y < logits.shape[1]:
    sel = tf.argmax(logits[0][y][x], axis=-1).numpy()
    rst.append(sel)
    if sel == 0:
      y += 1
    else:
      x += 1
  return moasseugi(vector_to_sentence([0] + list(filter(lambda x: x != 0, rst))))

In [156]:
datas = policy.dataset_extractor.get_value()

for data in datas:
  logits = policy.step_performer.transducer((tf.constant([data[0]], dtype=tf.float32), tf.constant([[0] + data[1]], dtype=tf.int32)))
  print(moasseugi(vector_to_sentence(data[1])))
  print(decode_with_hint(logits))
  print()

하 면허도 진짜 따야 되는데
먿ㅏ눛서노 단짜 나야 돼

아 이렇게 해서?
안ㅏ예렇게 수서?

동생이랑?
ㄷ뽕생이랑

아
안ㅓ

그래
개ㅡ

긴 하지
피ㅣ느하지

응 그냥
애ㅡ냐홰냥

데 가끔씩 막 비싼 데 가면은
내ㅐ 흐깜씩 막 비싼ㄷ데 왜련 ㅡ 

고 교양이었을 거 아냐
극ㅡ교양있 ㅓ을 거 아니

아
난ㅏ

근데 슈팅 게임이거든
게ㅔ  ㅔ 지기에하웜으

유튜브 편집자 해야지 뭐
유ㅠ초븐핕타 하 해야 ㅣ

백 명 단위야 꽤 많아
밴ㅣ 명 딴야야 겨 많아

음 에 맞아요
으ㅡ ㅇ괩맞아요

뭐 영화인데 어때?
모ㅗ 연뎌용데

노점상들 다 밀어버리니까 그냥
우ㅡ논댈  ㅏ 하 닐어버리니까 괘량

대신에 우리는 종업시를 늦게 하지
대ㅐ센에 우리는 점일이를 늦게 하지

강원도 동해
항ㅏ언도 동해

아
아ㅏ

그럴 수 있겠다
굴ㅡㅆㄹ 수 있겠다

아니 코가 더 비싸지
아ㅣ니 고가 더 비싸지

하여튼 그래서 어떻게 됐어? 뒤에가?
객ㅐㄷ튼 그래서 띠덯게 됐어  쥐에가

식민
지ㅡ맨ㅣ

다 먹은 줄 알
덩ㅗ뭑는 졸 알

나 우리 아빠 담배 끊은 게 너무 신기해
ㄴ만무리 아빠 뫼 깩끊은 제 시 ㅜ 힌기해

맞아
ㅁ맞아

겉 겉멋만 들어가지고?
거ㅘ ㅓ겉멋난 들어가 ㅣ구

어제 치킨을 먹긴 했거든?
오ㅓ딨치킨을 먹긴 했거든

얘 그리고 되게 금사빠야
이ㅣ 그 ㅐ   대게 그 사빠야

그건 너무 상대한테 못 할 짓이야
거ㅡ언 머무 상대한테 못 핮 짓이야

멋이 없잖아
머ㅏ르ㅣ 없잖아

아깝다 너는 진짜
아ㅏ ㅏ ㅜㅏ 저  ㄴ 잔

